## Getting the Data


In [ ]:
"""Import Libraries"""
import imaplib
import email
import getpass
import pandas as pd

In [ ]:
"""
Desired Email address is entered as a string
Then login to the email server
"""
username =  input("Enter your email address: ")
password = getpass.getpass("Enter password: ")
mail = imaplib.IMAP4_SSL('imap.gmail.com')


In [ ]:
mail.login(username, password)

In [ ]:
"""Checking the mailboxes and selecting one"""
print(mail.list())
mail.select("INBOX")


In [ ]:
"""Searching for emails in the mailbox"""
result, numbers = mail.search(None, '(FROM "quincy@freecodecamp.org")') 
uids1 = numbers[0].split()
uids = [id.decode("utf-8") for id in uids1 ]


In [ ]:
"""Fetching the desired content"""
result, messages = mail.fetch(','.join(uids) ,'(RFC822)') 

In [ ]:
body_list =[]
date_list = []
from_list = [] 
subject_list = []
for _, message in messages[::2]:
  email_message = email.message_from_bytes(message)
  email_subject = email.header.decode_header(email_message['Subject'])[0]
  for part in email_message.walk():
    print(part.get_content_type())
    if part.get_content_type() == "text/plain":
        body = part.get_payload(decode=True)
        body = body.decode("utf-8")
        body_list.append(body)
    else:
        continue
    if isinstance(email_subject[0],bytes):
      decoded = email_subject[0].decode(errors="ignore")
      subject_list.append(decoded)
    else:
      subject_list.append(email_subject[0])
  date_list.append(email_message.get('date'))
  fromlist = email_message.get('From')
  fromlist = fromlist.split("<")[0].replace('"', '')
  from_list.append(fromlist)


In [ ]:
date_list = pd.to_datetime(date_list)
date_list = [item.isoformat(' ')[:-6]for item in date_list]
print(len(subject_text))
print(len(from_list))
print(len(date_list))
print(len(body_list))
data = pd.DataFrame(data={'Date':date_list,'Sender':from_list,'Subject':subject_list, 'Body':body_list})
data.to_csv('emails.csv',index=False)

## DATA CLEANING

In [ ]:
data.head()

,Date,Sender,Subject,Body
0,2020-02-07 11:23:04,Quincy Larson,"I analyzed the results of a new survey of 116,...",Here are this week's five links that are worth...
1,2020-02-14 11:11:52,Quincy Larson,How to get your first job as a self-taught dev...,Here are this week's five links that are worth...
2,2020-02-21 11:41:13,Quincy Larson,"What I learned from 213,000 coding interview t...",Here are this week's five links that are worth...
3,2020-02-28 10:53:05,Quincy Larson,Resume tips from a developer who got job offer...,Here are this week's five links that are worth...
4,2020-03-06 10:28:17,Quincy Larson,freeCodeCamp just released a 7-hour Python Mac...,Here are this week's five links that are worth...


In [ ]:
def clean_data(data,column,i):
    data.loc[i, column] = data.loc[i, column].split("\r\n")
    new_string = " ".join(data.loc[i, column])
    new_string = new_string.split("'',")
    data[column][i:i+1] = pd.DataFrame(data = new_string)
    return data

In [ ]:
for n in range(len(data)):
    new_data = clean_data(data,"Body",n)

In [ ]:
new_data.head()